# 10.1 FinalAnalysis

Import the data set exported last script.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path = path = r'C:\Users\Neena Tilton\Dropbox\Projects\01_2020_InstacartBasket'

In [3]:
ords_prods_customers = pd.read_pickle(os.path.join(path, '02_Data', 'PreparedData', 'ords_prods_customers.pkl'))

In [6]:
ords_prods_customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 33 columns):
 #   Column                         Dtype   
---  ------                         -----   
 0   order_id                       int64   
 1   user_id                        int64   
 2   order_number                   int64   
 3   orders_day_of_week             int64   
 4   order_time_of_day              int64   
 5   days_since_prior_order         float64 
 6   product_id                     int64   
 7   add_to_cart_order              int64   
 8   reordered                      int64   
 9   product_name                   object  
 10  aisle_id                       int64   
 11  department_id                  int64   
 12  prices                         float64 
 13  _merge                         category
 14  busiest_day                    object  
 15  Busiest_days                   object  
 16  busiest_period_of_day          object  
 17  max order                

Considering security implications that might exist for most recent data. The only columns with PII data are 'first_name' and 'last_name'. Since customer names are not needed, creating a new dataframe with the two columns dropped. If we do, in some rare case, need to go back to trace the names of the customers, we will still have customer_id's to do so. 

In [7]:
df_names_dropped = ords_prods_customers.drop(columns = ['first_name', 'last_name'])

In [8]:
df_names_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 31 columns):
 #   Column                         Dtype   
---  ------                         -----   
 0   order_id                       int64   
 1   user_id                        int64   
 2   order_number                   int64   
 3   orders_day_of_week             int64   
 4   order_time_of_day              int64   
 5   days_since_prior_order         float64 
 6   product_id                     int64   
 7   add_to_cart_order              int64   
 8   reordered                      int64   
 9   product_name                   object  
 10  aisle_id                       int64   
 11  department_id                  int64   
 12  prices                         float64 
 13  _merge                         category
 14  busiest_day                    object  
 15  Busiest_days                   object  
 16  busiest_period_of_day          object  
 17  max order                

To investigate into customer behavior in different geographic areas, creating a regional segmentation of the data and categorized into 'region' in new column based on the 'state' column from your customers data set. 

In [10]:
# Creating lists for each region
Northeast_states = ['Maine', 'New Hamphire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 
                    'New York', 'Pennsylvania', 'New Jersey']

In [11]:
Midwest_states = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 
                  'Minnesota', 'Iowa', 'Missouri']

In [12]:
South_states = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina',
                'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 
                'Louisiana']

In [13]:
West_states = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington',
               'Oregon', 'California', 'Hawaii']

In [16]:
# Assign regional names as enteries in new column named 'region'
df_names_dropped.loc[df_names_dropped['state'].isin(West_states), 'region'] = 'West'

In [17]:
df_names_dropped.loc[df_names_dropped['state'].isin(South_states), 'region'] = 'South'

In [18]:
df_names_dropped.loc[df_names_dropped['state'].isin(Midwest_states), 'region'] = 'Midwest'

In [19]:
df_names_dropped.loc[df_names_dropped['state'].isin(Northeast_states), 'region'] = 'Northeast'

In [20]:
df_names_dropped['region'].value_counts(dropna = False)

South        10791885
West          8292913
Midwest       7597325
Northeast     5107358
NaN            615378
Name: region, dtype: int64

In [21]:
# There are some entries with no region name assigned. To address this, making a subset with NaN in the 'region' column
region_NaN_df = df_names_dropped[df_names_dropped['region'].isnull() == True]

In [23]:
region_NaN_df[['state', 'region']].head(100)

,state,region
17560,New Hampshire,NaN
17561,New Hampshire,NaN
17562,New Hampshire,NaN
17563,New Hampshire,NaN
17564,New Hampshire,NaN
...,...,...
20447,New Hampshire,NaN
20448,New Hampshire,NaN
20449,New Hampshire,NaN
20450,New Hampshire,NaN


In [24]:
# To see if New Hampshire is the only state in this subset dataframe with NaN assigned in 'region' column, using value count.
region_NaN_df['state'].value_counts(dropna = False)

New Hampshire    615378
Name: state, dtype: int64

In [26]:
# This confirms that there is a problem with the 'New Hampshire' spelling in the original list named Northeast_states.
# Originally it was misspelled as 'New Hamphire', fixing the spelling now.
Northeast_states = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 
                    'New York', 'Pennsylvania', 'New Jersey']

In [27]:
# Running loc() function again for Northeast region
df_names_dropped.loc[df_names_dropped['state'].isin(Northeast_states), 'region'] = 'Northeast'

In [28]:
# Checking once again if any missing values in 'region' column
df_names_dropped['region'].value_counts(dropna = False)

South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: region, dtype: int64

In [ ]:
# New 'region' column has been successfully created.

To determine whether there's a difference in spending habits between the different regions, creating crosstabs of the variables 'region' and 'spending flag' columns. 

In [57]:
crosstab = pd.crosstab(df_names_dropped['region'], df_names_dropped['spender_type_flag'], dropna = False)

In [58]:
crosstab

spender_type_flag,High spender,Low spender
region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


In [52]:
# NOTE: The numbers above are NOT YET separated nor grouped by user_id, so how many of each kind of spender is inaccurate since 
# every row is by order_id, which is each product purchased. 

Percent of transactions made by high spenders in each region:

In [59]:
mw_percent = 155975/(7441350+155975)

In [60]:
print(mw_percent)

0.020530252424373053


Midwest: 2.05%

In [61]:
ne_percent = 108225/(108225+5614511)

In [62]:
print(ne_percent)

0.018911408808653763


Northeast: 1.89%

In [63]:
s_percent = 209691/(209691+10582194)

In [64]:
print(s_percent)

0.019430433144904714


South: 1.94%

In [65]:
w_percent = 160354/(160354+8132559)

In [66]:
print(w_percent)

0.019336269414619446


West: 1.93%

The difference of spending habits between the regions is very small. Of the four regions, the Midwest had the highest proportion of transactions made by customers that categorized as 'High spenders.'

We want to only focus on customers who generate regular revenue for the app. Creating an exclusion flag for low-activity customers with less than 5 orders, and excluding them from the dataset. 

In [70]:
# We already have the column 'max_orders' showing the maximum order made by the user. With this, making a exclusion flag. 
df_names_dropped.loc[df_names_dropped['max_order'] < 5, 'exclusion_flag'] = 'Low Activity Customer'

In [71]:
df_names_dropped.loc[df_names_dropped['max_order'] >= 5, 'exclusion_flag'] = 'Regular Activity Customer'

In [72]:
df_names_dropped['exclusion_flag'].value_counts(dropna = False)

Regular Activity Customer    30964564
Low Activity Customer         1440295
Name: exclusion_flag, dtype: int64

In [73]:
# Creating a new dataframe with low-activity customers excluded
df_regular_activity = df_names_dropped[df_names_dropped['exclusion_flag'] == 'Regular Activity Customer']

In [74]:
# Checking to see if new dataframe only includes 'Regular Activity Customer'
df_regular_activity['exclusion_flag'].value_counts(dropna = False)

Regular Activity Customer    30964564
Name: exclusion_flag, dtype: int64

In [75]:
# Exporting this dataframe
df_regular_activity.to_pickle(os.path.join(path, '02_Data', 'PreparedData', 'reg_act_customers.pkl'))